# Proyecto Bimestral: Sistema de Recuperación de Información basado en Reuters-21578

## 1. Introducción
El objetivo de este proyecto es diseñar, construir, programar y desplegar un Sistema de Recuperación de Información (SRI) utilizando el corpus Reuters-21578. El proyecto se dividirá en varias fases,
que se describen a continuación.

In [1]:
import os
import zipfile
import pandas as pd
import string
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time

In [2]:
# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 2. Fases del Proyecto
### 2.1. Adquisición de Datos
- Objetivo: Obtener y preparar el corpus Reuters-21578.
- Tareas:
    - Descargar el corpus Reuters-21578.
    - Descomprimir y organizar los archivos.
    - Documentar el proceso de adquisición de datos

In [3]:
# Descargar y descomprimir el corpus Reuters-21578

def extract_reuters_data(zip_path, extract_to):
    """
    Extrae los datos del corpus Reuters-21578 desde un archivo zip a un directorio especificado.
    
    Parámetros:
        zip_path (str): Ruta al archivo zip que contiene los datos del corpus Reuters-21578.
        extract_to (str): Directorio de destino donde se descomprimirá el contenido del archivo zip.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("Datos descomprimidos en:", extract_to)

# Configuración de las rutas al archivo zip y al directorio de destino
zip_path = r"..\data\reuters.zip"
extract_to = r"..\data"

# Llamada a la función para extraer los datos
extract_reuters_data(zip_path, extract_to)

Datos descomprimidos en: ..\data


In [4]:
def change_extension_to_txt(folder_path):
    """
    Cambia la extensión de todos los archivos a .txt.
    
    Parámetros:
        folder_path (str): Ruta del directorio donde se encuentran los archivos.
    """
    if not os.path.exists(folder_path):
        print(f"La carpeta '{folder_path}' no existe. Verifica la ruta.")
        return

    for filename in os.listdir(folder_path):
        old_path = os.path.join(folder_path, filename)
        # Verificar si es un archivo regular y no una carpeta
        if os.path.isfile(old_path):
            # Cambiar la extensión a .txt
            new_filename = f"{filename}.txt" if '.' not in filename else f"{os.path.splitext(filename)[0]}.txt"
            new_path = os.path.join(folder_path, new_filename)
            os.rename(old_path, new_path)
            print(f"Archivo renombrado: {old_path} -> {new_path}")
        else:
            print(f"Omitido (no es un archivo): {old_path}")

# Ruta de la carpeta principal descomprimida
reuters_dir = r"..\data\reuters" 

# Cambiar extensiones en las carpetas training y test
training_dir = os.path.join(reuters_dir, "training")
test_dir = os.path.join(reuters_dir, "test")

print("Procesando carpeta 'training'...")
change_extension_to_txt(training_dir)

print("Procesando carpeta 'test'...")
change_extension_to_txt(test_dir)


Procesando carpeta 'training'...
Archivo renombrado: ..\data\reuters\training\1 -> ..\data\reuters\training\1.txt
Archivo renombrado: ..\data\reuters\training\10 -> ..\data\reuters\training\10.txt
Archivo renombrado: ..\data\reuters\training\100 -> ..\data\reuters\training\100.txt
Archivo renombrado: ..\data\reuters\training\1000 -> ..\data\reuters\training\1000.txt
Archivo renombrado: ..\data\reuters\training\10000 -> ..\data\reuters\training\10000.txt
Archivo renombrado: ..\data\reuters\training\10002 -> ..\data\reuters\training\10002.txt
Archivo renombrado: ..\data\reuters\training\10005 -> ..\data\reuters\training\10005.txt
Archivo renombrado: ..\data\reuters\training\10008 -> ..\data\reuters\training\10008.txt
Archivo renombrado: ..\data\reuters\training\10011 -> ..\data\reuters\training\10011.txt
Archivo renombrado: ..\data\reuters\training\10014 -> ..\data\reuters\training\10014.txt
Archivo renombrado: ..\data\reuters\training\10015 -> ..\data\reuters\training\10015.txt
Archivo 

In [5]:
def parse_cats_file(cats_file_path):
    """
    Lee el archivo 'cats.txt' y crea un diccionario que asocia a cada par de categorías
    (origen, nombre) su lista correspondiente de categorías.

    Parámetros:
        cats_file_path (str): Ruta al archivo 'cats.txt' que contiene las categorías y sus asociaciones.

    Retorna:
        dict: Un diccionario donde las claves son tuplas (origen, nombre) y los valores son las categorías asociadas como cadenas de texto.
    
    """
    categories = {}
    with open(cats_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()  # Dividimos cada línea por espacios
            origin_and_name = parts[0]  # Tomamos la primera parte (origen/nombre)
            origin, name = origin_and_name.split('/')  # Separamos origen y nombre por '/'
            category_list = " ".join(parts[1:])  # El resto son las categorías asociadas
            categories[(origin, name)] = category_list  # Guardamos en el diccionario
    return categories


In [6]:
def extract_document_info(folder_path, origin, categories_dict):
    """
    Extrae la información relevante de los documentos dentro de una carpeta.

    La función recorre todos los archivos de texto (.txt) dentro de una carpeta especificada, lee su contenido y extrae el título, 
    el contenido y la categoría asociada a cada documento, la cual se obtiene del diccionario de categorías proporcionado.

    Parámetros:
        folder_path (str): Ruta a la carpeta que contiene los documentos de texto.
        origin (str): El origen del documento, utilizado para buscar la categoría correspondiente en el diccionario.
        categories_dict (dict): Diccionario que asocia a cada par (origen, nombre de archivo) con su categoría.

    Retorna:
        list: Una lista de diccionarios, cada uno con la información de un documento (nombre, título, contenido, origen y categoría).

    """
    documents_data = []  # Lista para almacenar los datos de los documentos procesados

    # Recorrer todos los archivos en la carpeta
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)  # Obtener la ruta completa del archivo

        # Verificar que el archivo sea un archivo de texto (.txt)
        if os.path.isfile(file_path) and filename.endswith('.txt'):
            
            # Abrir el archivo y leer su contenido
            with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                lines = f.readlines()  # Leer todas las líneas del archivo

                # Extraer el título (primera línea del archivo) y el contenido (resto del archivo)
                title = lines[0].strip() if lines else ''  # Asignar título si el archivo no está vacío
                content = "".join(lines[1:]).strip()  # Unir el resto de las líneas para el contenido

                # Buscar la categoría del documento en el diccionario usando el origen y el nombre del archivo (sin extensión)
                category = categories_dict.get((origin, filename.split('.')[0]), '')

                # Añadir la información del documento a la lista
                documents_data.append({
                    'Nombre': filename.split('.')[0],  # El nombre del documento sin la extensión
                    'Titulo': title,  # El título extraído
                    'Contenido': content,  # El contenido extraído
                    'Origen': origin,  # El origen proporcionado
                    'Categoria': category  # La categoría obtenida del diccionario
                })
    
    # Devolver la lista con los datos de todos los documentos procesados
    return documents_data


## 2.2. Preprocesamiento
### Objetivo: Limpiar y preparar los datos para su análisis.
- Tareas:
    - Extraer el contenido relevante de los documentos.
    - Realizar limpieza de datos: eliminación de caracteres no deseados, normalización de texto, etc.
    - Tokenización: dividir el texto en palabras o tokens.
    - Eliminar stop words y aplicar stemming o lematización.

In [7]:
def clean_text(text):
    """
    Elimina caracteres no deseados y normaliza el texto

    La función realiza las siguientes tareas de preprocesamiento en el texto:
    - Convierte todo el texto a minúsculas.
    - Elimina la puntuación utilizando el módulo 'string.punctuation'.
    - Elimina los espacios iniciales y finales del texto.

    Parámetros:
        text (str): El texto que se va a limpiar.

    Retorna:
        str: El texto limpio y normalizado

    """
    text = text.lower()  # Convertir a minúsculas 
    text = text.translate(str.maketrans('', '', string.punctuation))  # Eliminar la puntuación
    text = text.strip()  # Eliminar espacios iniciales y finales
    return text


In [8]:
# Diccionario de normalización (abreviaturas a términos completos)
normalization_dict = {
    # Abreviaciones de países y regiones
    "usa": "united states",
    "u.s.": "united states",
    "uk": "united kingdom",
    "u.k.": "united kingdom",
    "eu": "european union",
    "e.u.": "european union",
    "br": "brazil",
    "aus": "australia",
    "can": "canada",
    "germany": "germany",
    "fr": "france",
    "ind": "india",
    "ita": "italy",
    "jpn": "japan",
    "mex": "mexico",
    "rus": "russia",
    "ch": "china",
    "nzl": "new zealand",
    "saf": "south africa",
    "saudi arabia": "saudi arabia",
    "singapore": "singapore",
    
    # Abreviaturas de organizaciones y empresas
    "bbc": "british broadcasting corporation",
    "cia": "central intelligence agency",
    "euroland": "european union",
    "opec": "organization of petroleum exporting countries",
    "world bank": "world bank",
    "imf": "international monetary fund",
    "fbi": "federal bureau of investigation",
    "un": "united nations",
    "nato": "north atlantic treaty organization",
    "wto": "world trade organization",
    "g7": "group of seven",
    "g20": "group of twenty",
    "nasdaq": "national association of securities dealers automated quotations",
    "nyse": "new york stock exchange",
    "ftse": "financial times stock exchange",
    "dow": "dow jones industrial average",
    "apple": "apple inc.",
    "google": "google inc.",
    "microsoft": "microsoft corporation",
    "amazon": "amazon.com, inc.",
    "facebook": "facebook, inc.",
    "tesla": "tesla, inc.",
    
    # Abreviaturas de títulos y personas
    "mr": "mister",
    "mrs": "missus",
    "ms": "miss",
    "dr": "doctor",
    "prof": "professor",
    "pres": "president",
    "govt": "government",
    "ceo": "chief executive officer",
    "cfo": "chief financial officer",
    "cto": "chief technology officer",
    "coo": "chief operating officer",
    
    # Abreviaturas de términos financieros y económicos
    "gdp": "gross domestic product",
    "gni": "gross national income",
    "fdi": "foreign direct investment",
    "npl": "non-performing loan",
    "loi": "letter of intent",
    "ipo": "initial public offering",
    "m&a": "mergers and acquisitions",
    "equity": "equity",
    "debt": "debt",
    "stock market": "stock market",
    "bonds": "bonds",
    "lbo": "leveraged buyout",
    "ebitda": "earnings before interest, taxes, depreciation, and amortization",
    "cash flow": "cash flow",
    "earnings": "earnings",
    "revenue": "revenue",
    
    # Abreviaturas de tecnología
    "it": "information technology",
    "ai": "artificial intelligence",
    "ml": "machine learning",
    "iot": "internet of things",
    "big data": "big data",
    "cloud computing": "cloud computing",
    "saas": "software as a service",
    "paas": "platform as a service",
    "dba": "database administrator",
    "cybersecurity": "cybersecurity",
    "vpn": "virtual private network",
    "url": "uniform resource locator",
    
    # Términos generales y otros
    "e.g.": "for example",
    "i.e.": "that is",
    "vs": "versus",
    "etc": "et cetera",
    "no.": "number",
    "p.m.": "prime minister",
    "b.c.": "before christ",
    "a.d.": "anno domini",
    "usd": "united states dollar",
    "eur": "euro",
    "gbp": "great british pound",
    "inr": "indian rupee",
    "jpy": "japanese yen",
    "myr": "malaysian ringgit",
    "czk": "czech koruna",
    "krw": "south korean won",
    "sgd": "singapore dollar",
    "chf": "swiss franc",
    "nzd": "new zealand dollar",
    "xrp": "ripple",
    "btc": "bitcoin",
    "eth": "ethereum",
    
    # Términos de agencias de noticias y eventos
    "reuters": "reuters news agency",
    "bloomberg": "bloomberg",
    "bbc": "british broadcasting corporation",
    "cnn": "cable news network",
    "ft": "financial times",
    "wsj": "wall street journal",
    "nytimes": "new york times",
    "forbes": "forbes magazine",
    "cnbc": "cnbc",
    "wsj": "wall street journal",
    "ft": "financial times",
    
    # Otras abreviaturas comunes
    "inc.": "incorporated",
    "llc": "limited liability company",
    "co.": "company",
    "corp.": "corporation",
    "ltd": "limited",
    "co-op": "cooperative",
    "univ": "university",
    "dept": "department",
    "edu": "education",
    "gov": "government",
    "yr": "year",
    "mo": "month",
    "wks": "weeks",
    "hrs": "hours",
    "min": "minutes",
    "sec": "seconds"
}

In [9]:
def normalize_text(text, normalization_dict):
    """
    Normaliza el texto reemplazando abreviaciones y términos por sus formas completas.
    
    Parámetros:
        text (str): El texto que se va a normalizar.
        normalization_dict (dict): Diccionario con abreviaciones y sus formas completas.
    
    Retorna:
        str: El texto normalizado.
    """
    words = text.split()  # Separa el texto en palabras
    normalized_words = [normalization_dict.get(word, word) for word in words]  # Reemplaza según el diccionario
    return " ".join(normalized_words)  # Devuelve el texto normalizado


In [10]:

def preprocess_text(content, normalization_dict):
    """ 
    Realiza la limpieza, tokenización, eliminación de stopwords, stemming y normalización del texto.

    La función aplica una serie de pasos de preprocesamiento al texto:
    1. Limpieza del texto: elimina caracteres no deseados y normaliza el texto.
    2. Tokenización: divide el texto en palabras o tokens.
    3. Eliminación de stopwords: filtra las palabras irrelevantes.
    4. Aplicación de stemming: reduce las palabras a su raíz.
    5. Normalización: reemplaza abreviaciones y términos por sus formas completas.

    Parámetros:
        content (str): El texto que se va a preprocesar.
        normalization_dict (dict): Diccionario con abreviaciones y sus formas completas.

    Retorna:
        str: El texto preprocesado, con las palabras lematizadas, sin stopwords, normalizado y listo para el análisis.

    """
    # Paso 1: Limpieza de texto - Se normaliza el texto eliminando puntuación y convirtiendo a minúsculas.
    cleaned_text = clean_text(content)
    
    # Paso 2: Tokenización - Dividimos el texto limpio en palabras o tokens.
    tokens = word_tokenize(cleaned_text)
    
    # Paso 3: Eliminación de stopwords - Eliminamos las palabras comunes y sin significado relevante para el análisis.
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Paso 4: Stemming - Reducimos las palabras a su raíz usando el algoritmo PorterStemmer.
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    
    # Paso 5: Reconstrucción del texto preprocesado - Unimos los tokens procesados en una cadena de texto.
    preprocessed_text = " ".join(stemmed_tokens)
    
    # Paso 6: Normalización - Reemplazamos abreviaciones y términos por sus formas completas usando el diccionario.
    normalized_text = normalize_text(preprocessed_text, normalization_dict)
    
    return normalized_text



In [11]:
# Preprocesamiento de documentos
def preprocess_documents(data):
    """
    Aplica preprocesamiento al contenido de cada documento en los datos.

    La función recorre cada documento en el conjunto de datos, preprocesando tanto el contenido 
    como el título del documento. El contenido y el título pasan por la función `preprocess_text` 
    para realizar limpieza, tokenización, eliminación de stopwords, stemming y normalización.

    Parámetros:
        data (list): Una lista de diccionarios donde cada diccionario representa un documento con claves 
                     como 'Contenido' (texto del documento) y 'Titulo' (título del documento).

    Retorna:
        list: La lista de documentos con los campos 'Contenido Preprocesado' y 'Titulo Preprocesado' 
              añadidos, que contienen el texto limpio y normalizado de cada documento.

    """
    # Recorremos cada documento en los datos para aplicar el preprocesamiento
    for doc in data:
        original_content = doc['Contenido']  # Extraemos el contenido original del documento
        preprocessed_content = preprocess_text(original_content, normalization_dict)  # Preprocesamos el contenido
        doc['Contenido_Preprocesado'] = preprocessed_content  # Guardamos el contenido preprocesado

        original_title = doc['Titulo']  # Extraemos el título original del documento
        preprocessed_title = preprocess_text(original_title, normalization_dict)  # Preprocesamos el título
        doc['Titulo_Preprocesado'] = preprocessed_title  # Guardamos el título preprocesado

    return data  # Devolvemos la lista de documentos con los campos preprocesados


In [12]:
# Rutas
training_dir = os.path.join(reuters_dir, "training")  # Ruta al directorio de entrenamiento
test_dir = os.path.join(reuters_dir, "test")  # Ruta al directorio de prueba
cats_file_path = os.path.join(reuters_dir, "cats.txt")  # Ruta al archivo de categorías

# Leer el archivo cats.txt para obtener las categorías
categories_dict = parse_cats_file(cats_file_path)  # Procesa el archivo cats.txt para obtener el diccionario de categorías

# Procesar carpetas training y test
training_data = extract_document_info(training_dir, "training", categories_dict)  # Extrae la información de los documentos en el directorio de entrenamiento
test_data = extract_document_info(test_dir, "test", categories_dict)  # Extrae la información de los documentos en el directorio de prueba

# Preprocesar el contenido de los documentos
all_data = training_data + test_data  # Combina los datos de entrenamiento y prueba
all_data = preprocess_documents(all_data)  # Aplica el preprocesamiento a todo el contenido de los documentos (títulos y cuerpos de los documentos)

# Guardar en un archivo Excel
df = pd.DataFrame(all_data)  # Convierte la lista de diccionarios en un DataFrame de pandas
output_excel_path = os.path.join(reuters_dir, "reuters_data_preprocessed.xlsx")  # Define la ruta de salida del archivo Excel
df.to_excel(output_excel_path, index=False)  # Guarda el DataFrame en un archivo Excel sin incluir el índice de filas


## 2.3. Representacion de Datos en Espacio Vectorial
### Objetivo: Convertir los textos en una forma que los algoritmos puedan procesar.
- Tareas:
    - Utilizar técnicas como Bag ofWords (BoW), TF-IDF, yWord2Vec para vectorizar el texto.
    - Evaluar las diferentes técnicas de vectorización.
    - Documentar los métodos y resultados obtenidos.

In [13]:
# Ruta de la carpeta principal descomprimida
reuters_dir = r"..\data\reuters"

# Ruta al archivo Excel preprocesado
input_excel_path = os.path.join(reuters_dir, "reuters_data_preprocessed.xlsx")  # Define la ruta donde se encuentra el archivo Excel con los datos preprocesados

# Leer el archivo Excel usando pandas
df = pd.read_excel(input_excel_path)  # Carga el archivo Excel en un DataFrame de pandas

# Seleccionar el contenido preprocesado
texts = df['Contenido_Preprocesado'].fillna("").tolist()  # Extrae la columna 'Contenido Preprocesado', reemplaza valores nulos con cadenas vacías y convierte la columna a una lista de textos


In [14]:
# Bag of Words (BoW)
def bag_of_words(texts):
    """
    Convierte una lista de textos en una matriz de características utilizando el modelo Bag of Words (BoW).

    La función utiliza el `CountVectorizer` de scikit-learn para crear una representación numérica de los textos,
    donde cada columna corresponde a una palabra única (vocabulario) y cada fila representa un documento.

    Parámetros:
        texts (list): Lista de cadenas de texto que se van a convertir en una matriz BoW.

    Retorna:
        tuple: 
            - bow_matrix (sparse matrix): Matriz dispersa que representa los textos como vectores de frecuencia de palabras.
            - bow_features (array): Lista de palabras (características) que forman el vocabulario.
    """
    # Crear el vectorizador de palabras (BoW)
    vectorizer = CountVectorizer()  # Inicializa el CountVectorizer
    
    # Convertir los textos en una matriz de características BoW
    bow_matrix = vectorizer.fit_transform(texts)  # Ajusta el vectorizador a los textos y genera la matriz
    
    # Obtener las características (palabras) que forman el vocabulario
    bow_features = vectorizer.get_feature_names_out()  # Extrae las palabras del vocabulario
    
    # Imprimir el tamaño de la matriz BoW
    print(f"BoW: Matriz de tamaño {bow_matrix.shape}")  # Imprime las dimensiones de la matriz resultante
    
    return bow_matrix, bow_features  # Devuelve la matriz BoW y el vocabulario


In [15]:
# TF-IDF
def tf_idf(texts):
    """
    Convierte una lista de textos en una matriz de características utilizando el modelo TF-IDF (Term Frequency - Inverse Document Frequency).
    
    La función utiliza el `TfidfVectorizer` de scikit-learn para transformar los textos en una representación numérica, 
    donde cada columna corresponde a una palabra única del vocabulario, y cada fila representa un documento con un peso 
    que indica la importancia relativa de la palabra en el documento con respecto al corpus completo.

    Parámetros:
        texts (list): Lista de cadenas de texto que se van a convertir en una matriz TF-IDF.

    Retorna:
        tuple: 
            - tfidf_matrix (sparse matrix): Matriz dispersa que representa los textos como vectores de peso de palabras.
            - tfidf_features (array): Lista de palabras (características) que forman el vocabulario.
    """
    # Crear el vectorizador TF-IDF
    vectorizer = TfidfVectorizer()  # Inicializa el TfidfVectorizer
    
    # Convertir los textos en una matriz de características TF-IDF
    tfidf_matrix = vectorizer.fit_transform(texts)  # Ajusta el vectorizador a los textos y genera la matriz
    
    # Obtener las características (palabras) que forman el vocabulario
    tfidf_features = vectorizer.get_feature_names_out()  # Extrae las palabras del vocabulario
    
    # Imprimir el tamaño de la matriz TF-IDF
    print(f"TF-IDF: Matriz de tamaño {tfidf_matrix.shape}")  # Imprime las dimensiones de la matriz resultante
    
    return tfidf_matrix, tfidf_features  # Devuelve la matriz TF-IDF y el vocabulario


In [16]:
# Word2Vec
def word2vec(texts, vector_size=100, window=5, min_count=1):
    """
    Genera representaciones vectoriales de palabras utilizando el modelo Word2Vec.

    La función usa la biblioteca `gensim` para crear un modelo Word2Vec, que convierte cada palabra en los textos 
    proporcionados en un vector numérico de características, capturando las relaciones semánticas entre palabras.

    Parámetros:
        texts (list): Lista de cadenas de texto que serán tokenizadas (separadas en palabras) y procesadas.
        vector_size (int): Tamaño de los vectores de características para cada palabra. Default es 100.
        window (int): Número de palabras contextuales a considerar alrededor de cada palabra en el modelo. Default es 5.
        min_count (int): Número mínimo de apariciones de una palabra para ser incluida en el modelo. Default es 1.

    Retorna:
        word_vectors: Objeto `KeyedVectors` que contiene las representaciones vectoriales de las palabras.

    """
    # Tokenización: Se convierte cada texto en una lista de palabras (tokens).
    tokenized_texts = [text.split() for text in texts]  # Separa cada texto en palabras
    
    # Entrenamiento del modelo Word2Vec con los textos tokenizados.
    model = Word2Vec(sentences=tokenized_texts, vector_size=vector_size, window=window, min_count=min_count)  # Ajusta el modelo
    
    # Obtener los vectores de palabras entrenados.
    word_vectors = model.wv  # Extrae las representaciones vectoriales de las palabras
    
    # Imprimir el número de palabras y el tamaño de los vectores.
    print(f"Word2Vec: {len(word_vectors)} palabras representadas con vectores de tamaño {vector_size}")  # Muestra la cantidad de palabras
    
    return word_vectors  # Devuelve las representaciones vectoriales de las palabras


In [17]:
# Generar representaciones

# Paso 1: Aplicar el modelo Bag of Words (BoW) para convertir los textos en una matriz de frecuencias de palabras
bow_matrix, bow_features = bag_of_words(texts)  # Llama a la función bag_of_words para obtener la matriz BoW y las características

# Paso 2: Aplicar el modelo TF-IDF para convertir los textos en una matriz de pesos de palabras
tfidf_matrix, tfidf_features = tf_idf(texts)  # Llama a la función tf_idf para obtener la matriz TF-IDF y las características

# Paso 3: Aplicar el modelo Word2Vec para obtener representaciones vectoriales de las palabras
word_vectors = word2vec(texts)  # Llama a la función word2vec para generar los vectores de palabras


BoW: Matriz de tamaño (10789, 38331)
TF-IDF: Matriz de tamaño (10789, 38331)
Word2Vec: 38359 palabras representadas con vectores de tamaño 100


In [18]:
# Documentar resultados
# Se crea un diccionario para almacenar los resultados de cada técnica de representación
results = {
    "Técnica": ["Bag of Words", "TF-IDF", "Word2Vec"],  # Lista con los nombres de las técnicas aplicadas
    "Dimensión de Matriz": [bow_matrix.shape, tfidf_matrix.shape, len(word_vectors)],  # Tamaño de la matriz generada por cada técnica
    "Tamaño de Vocabulario": [len(bow_features), len(tfidf_features), len(word_vectors)]  # Número de características (palabras) en el vocabulario de cada técnica
}

# Crear DataFrame con resultados
# Se convierte el diccionario de resultados en un DataFrame de pandas para mejor visualización
results_df = pd.DataFrame(results)  # Crea un DataFrame con los resultados para fácil acceso y análisis


In [19]:
# Función para preprocesar la consulta
def preprocess_query(query, normalization_dict):
    """
    Aplica el preprocesamiento a la consulta, igual que se hace para el corpus.
    """
    preprocessed_query = preprocess_text(query, normalization_dict)  # Se usa el mismo preprocesamiento aplicado al corpus
    return preprocessed_query

In [20]:
# Función para medir el tiempo de ejecución de cada técnica
def evaluate_vectorization(query, normalization_dict, bow_vectorizer, tfidf_vectorizer, word2vec_model, num_runs=10):
    """
    Evalúa el tiempo de ejecución de cada técnica de vectorización (BoW, TF-IDF, Word2Vec) para la consulta dada,
    ejecutado múltiples veces para tomar un tiempo promedio.
    """
    # Preprocesar la consulta
    preprocessed_query = preprocess_query(query, normalization_dict)
    
    # Variables para almacenar los tiempos
    bow_times = []
    tfidf_times = []
    word2vec_times = []
    
    # Ejecutar las evaluaciones múltiples veces
    for _ in range(num_runs):
        # Evaluar con Bag of Words (BoW)
        start_time = time.time()
        bow_vector = bow_vectorizer.transform([preprocessed_query])  # Convertir consulta a BoW
        bow_times.append(time.time() - start_time)  # Medir tiempo
        
        # Evaluar con TF-IDF
        start_time = time.time()
        tfidf_vector = tfidf_vectorizer.transform([preprocessed_query])  # Convertir consulta a TF-IDF
        tfidf_times.append(time.time() - start_time)  # Medir tiempo
        
        # Evaluar con Word2Vec
        start_time = time.time()
        word2vec_vector = [word2vec_model[word] for word in preprocessed_query.split() if word in word2vec_model]  # Convertir consulta a Word2Vec
        word2vec_times.append(time.time() - start_time)  # Medir tiempo
    
    # Promedio de los tiempos
    avg_bow_time = np.mean(bow_times)
    avg_tfidf_time = np.mean(tfidf_times)
    avg_word2vec_time = np.mean(word2vec_times)
    
    # Mostrar los tiempos de ejecución promedio para cada técnica
    print(f"Tiempo promedio de ejecución para Bag of Words: {avg_bow_time:.8f} segundos")
    print(f"Tiempo promedio de ejecución para TF-IDF: {avg_tfidf_time:.8f} segundos")
    print(f"Tiempo promedio de ejecución para Word2Vec: {avg_word2vec_time:.8f} segundos")
    
    # Devolver los resultados promedio
    return {
        "BoW Time": avg_bow_time,
        "TF-IDF Time": avg_tfidf_time,
        "Word2Vec Time": avg_word2vec_time
    }

In [21]:
query = "the farmer-owned reserve national five-day"

# Inicializar los vectorizadores
bow_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()

# Ajustar los vectorizadores a los datos de entrenamiento 
bow_vectorizer.fit(texts)  # Ajusta BoW al corpus
tfidf_vectorizer.fit(texts)  # Ajusta TF-IDF al corpus

# Crear el modelo Word2Vec
tokenized_texts = [text.split() for text in texts]  # Tokenizar los textos
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1)

# Evaluar la consulta
results = evaluate_vectorization(query, normalization_dict, bow_vectorizer, tfidf_vectorizer, word2vec_model.wv, num_runs=10)

# Mostrar los resultados
print(results)


Tiempo promedio de ejecución para Bag of Words: 0.00010307 segundos
Tiempo promedio de ejecución para TF-IDF: 0.00091810 segundos
Tiempo promedio de ejecución para Word2Vec: 0.00000000 segundos
{'BoW Time': 0.00010306835174560547, 'TF-IDF Time': 0.0009181022644042969, 'Word2Vec Time': 0.0}


## 2.4. Indexación
### Objetivo: Crear un índice que permita búsquedas eficientes.
- Tareas:
    - Construir un índice invertido que mapee términos a documentos.
    - Implementar y optimizar estructuras de datos para el índice.
    - Documentar el proceso de construcción del índice.

In [22]:
def build_inverted_index(documents):
    """
    Construye un índice invertido que mapea términos a documentos.
    
    Parámetros:
        documents (list): Lista de diccionarios con los datos de los documentos.
    
    Retorna:
        dict: Índice invertido donde las claves son términos y los valores son listas de documentos.
    """
    # Crear un diccionario para el índice invertido donde cada término apunta a un conjunto de IDs de documentos.
    inverted_index = defaultdict(set)  # Usamos un set para evitar duplicados
    
    # Recorrer cada documento en la lista de documentos
    for doc in documents:
        doc_id = doc['Nombre']  # Obtener el ID único del documento
        content = doc['Contenido_Preprocesado']  # Obtener el contenido preprocesado del documento
        terms = set(content.split())  # Tokenizar el contenido en términos únicos (sin repetir palabras)

        # Para cada término en el documento, agregar el ID del documento al índice invertido
        for term in terms:
            inverted_index[term].add(doc_id)  # Asociar el término con el documento correspondiente
    
    # Convertir los sets a listas para que el índice invertido sea más fácil de manejar
    return {term: list(doc_ids) for term, doc_ids in inverted_index.items()}  # Devolver el índice invertido como un diccionario


In [23]:
def save_inverted_index_to_excel(inverted_index, output_path):
    """
    Guarda el índice invertido en un archivo Excel para fácil visualización.
    
    Parámetros:
        inverted_index (dict): Índice invertido.
        output_path (str): Ruta del archivo Excel donde se guardará.
    """
    # Crear una lista de diccionarios con los términos y sus documentos correspondientes.
    # Cada diccionario tendrá el término y una cadena con los IDs de los documentos donde aparece ese término.
    index_data = [{"Término": term, "Documentos": ", ".join(map(str, doc_ids))} for term, doc_ids in inverted_index.items()]

    
    # Convertir la lista de diccionarios en un DataFrame de pandas para facilitar la exportación.
    df = pd.DataFrame(index_data)
    
    # Guardar el DataFrame en un archivo Excel sin incluir el índice de filas.
    df.to_excel(output_path, index=False)
    
    # Imprimir mensaje confirmando la ubicación del archivo guardado.
    print(f"Índice invertido guardado en: {output_path}")


In [24]:
# Seleccionar los datos preprocesados
documents = df.to_dict(orient='records')

# Construir índice invertido
inverted_index = build_inverted_index(documents)

# Guardar resultados en Excel
output_excel_path = os.path.join(reuters_dir, "inverted_index.xlsx")
save_inverted_index_to_excel(inverted_index, output_excel_path)

# Guardar resultados en Excel en el back
inverted_index_back_path = r'..\back'
output_excel_pathb = os.path.join(inverted_index_back_path, "inverted_index.xlsx")
save_inverted_index_to_excel(inverted_index, output_excel_pathb)

# Documentación del proceso
print("Índice invertido creado con éxito.")
print(f"Términos indexados: {len(inverted_index)}")

Índice invertido guardado en: ..\data\reuters\inverted_index.xlsx
Índice invertido guardado en: ..\back\inverted_index.xlsx
Índice invertido creado con éxito.
Términos indexados: 38359


## 2.5. Diseño del Motor de Búsqueda
### Objetivo: Implementar la funcionalidad de búsqueda.
- Tareas:
    - Desarrollar la lógica para procesar consultas de usuarios.
    - Utilizar algoritmos de similitud como similitud coseno o Jaccard.
    - Desarrollar un algoritmo de ranking para ordenar los resultados.
    - Documentar la arquitectura y los algoritmos utilizados.

In [25]:
# Preprocesamiento de consulta
def preprocess_query(query, stop_words):
    """
    Limpia y preprocesa la consulta ingresada por el usuario.
    
    Parámetros:
        query (str): La consulta ingresada por el usuario.
        stop_words (list): Lista de palabras vacías (stopwords) a eliminar de la consulta.

    Retorna:
        list: Lista de tokens filtrados y procesados de la consulta.
    """
    # Limpiar la consulta: convertir a minúsculas y eliminar puntuación
    query = query.lower().translate(str.maketrans('', '', string.punctuation))  

    # Tokenizar la consulta
    tokens = query.split()  

    # Eliminar las stop words
    tokens = [word for word in tokens if word not in stop_words]  

    return tokens  # Devuelve la lista de palabras procesadas

# Cargar datos del índice invertido y documentos
input_excel_path = os.path.join(reuters_dir, "reuters_data_preprocessed.xlsx")
df = pd.read_excel(input_excel_path)  # Lee el archivo Excel con los datos preprocesados

# Extraer los documentos y sus IDs
documents = df['Contenido_Preprocesado'].tolist()  # Obtiene la lista de contenidos preprocesados
document_ids = df['Nombre'].tolist()  # Obtiene la lista de nombres de los documentos


In [26]:
# Vectorización con TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

def search_query_cosine(query, tfidf_vectorizer, tfidf_matrix, document_ids, top_k=10):
    """
    Realiza una búsqueda utilizando similitud coseno.
    """
    query_vector = tfidf_vectorizer.transform([query])  # Vectorizar consulta
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Ordenar documentos por similitud descendente
    ranked_indices = np.argsort(-cosine_similarities)[:top_k]
    results = [(document_ids[i], cosine_similarities[i]) for i in ranked_indices if cosine_similarities[i] > 0]
    return results

In [27]:
# Vectorización con TF-IDF
tfidf_vectorizer = TfidfVectorizer()  # Inicializa el vectorizador TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)  # Aplica el vectorizador a los documentos

def search_query_cosine(query, tfidf_vectorizer, tfidf_matrix, document_ids, top_k=10):
    """
    Realiza una búsqueda utilizando similitud coseno para encontrar los documentos más relevantes en función de una consulta.
    
    La función toma una consulta de texto y calcula su similitud coseno con los documentos preprocesados y vectorizados
    utilizando el modelo TF-IDF. Devuelve los documentos más similares junto con sus puntuaciones de similitud.
    
    Parámetros:
        query (str): La consulta de texto ingresada por el usuario.
        tfidf_vectorizer (TfidfVectorizer): El vectorizador TF-IDF previamente ajustado.
        tfidf_matrix (sparse matrix): La matriz de características de TF-IDF de los documentos.
        document_ids (list): Lista con los IDs de los documentos.
        top_k (int): Número de resultados más relevantes a devolver. Default es 10.
    
    Retorna:
        list: Una lista de tuplas donde cada tupla contiene un ID de documento y su puntuación de similitud coseno.
    """
    # Convertir la consulta a un vector TF-IDF
    query_vector = tfidf_vectorizer.transform([query])  

    # Calcular la similitud coseno entre la consulta y todos los documentos
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()  

    # Ordenar los documentos por similitud coseno en orden descendente
    ranked_indices = np.argsort(-cosine_similarities)[:top_k]  

    # Filtrar los resultados y devolver solo aquellos con puntuación positiva
    results = [(document_ids[i], cosine_similarities[i]) for i in ranked_indices if cosine_similarities[i] > 0]
    
    return results  # Devuelve los documentos más similares junto con sus puntuaciones de similitud


In [28]:
def jaccard_similarity(query_tokens, doc_tokens):
    """
    Calcula la similitud de Jaccard entre la consulta y un documento.

    Parámetros:
        query_tokens (list): Lista de tokens (palabras) de la consulta preprocesada.
        doc_tokens (list): Lista de tokens (palabras) del documento preprocesado.

    Retorna:
        float: El valor de la similitud de Jaccard entre la consulta y el documento. Un valor entre 0 y 1,
               donde 1 significa que los conjuntos son idénticos y 0 significa que no comparten ningún término.
    """
    # Calcular la intersección de los conjuntos de tokens
    intersection = len(set(query_tokens).intersection(set(doc_tokens))) 

    # Calcular la unión de los conjuntos de tokens
    union = len(set(query_tokens).union(set(doc_tokens)))  

    # Retornar la similitud de Jaccard (intersección / unión)
    return intersection / union  


In [29]:
def search_query_jaccard(query, documents, document_ids, top_k=10):
    """
    Realiza una búsqueda utilizando el coeficiente de Jaccard para encontrar los documentos más relevantes en función de una consulta.
    
    Parámetros:
        query (str): La consulta ingresada por el usuario, como una cadena de texto.
        documents (list): Lista de textos de los documentos preprocesados.
        document_ids (list): Lista de identificadores de los documentos correspondientes.
        top_k (int): Número de resultados más relevantes a devolver. El valor por defecto es 10.

    Retorna:
        list: Una lista de tuplas, donde cada tupla contiene un ID de documento y su puntuación de similitud de Jaccard.

    """
    results = []  # Lista para almacenar los resultados de similitud
    
    # Tokenizar la consulta en palabras
    query_tokens = query.split()
    
    # Iterar sobre cada documento y su ID correspondiente
    for doc_id, doc_content in zip(document_ids, documents):
        doc_tokens = doc_content.split()  # Tokenizar el contenido del documento
        score = jaccard_similarity(query_tokens, doc_tokens)  # Calcular la similitud de Jaccard entre consulta y documento
        
        # Si la similitud es mayor a 0, agregar el documento y su puntuación a los resultados
        if score > 0:
            results.append((doc_id, score))
    
    # Ordenar los resultados por la puntuación de similitud en orden descendente y seleccionar los primeros 'top_k' resultados
    results = sorted(results, key=lambda x: x[1], reverse=True)[:top_k]
    
    # Retornar los documentos más relevantes
    return results


In [30]:
def rank_results(results, method="cosine"):
    """
    Ordena los resultados de búsqueda según el método de clasificación especificado.
    
    Parámetros:
        results (list): Lista de tuplas, donde cada tupla contiene un ID de documento y su puntuación.
        method (str): El método de clasificación a usar.

    Retorna:
        list: La lista de resultados ordenada en orden descendente según las puntuaciones.

    """
    # Ordenar los resultados por la puntuación (segundo valor de cada tupla) en orden descendente
    return sorted(results, key=lambda x: x[1], reverse=True)


In [31]:
# Probar con una consulta
user_query = "BAHIA COCOA REVIEW"
preprocessed_query = " ".join(preprocess_query(user_query, set(stopwords.words('english'))))

In [32]:
# Búsqueda con similitud coseno
cosine_results = search_query_cosine(preprocessed_query, tfidf_vectorizer, tfidf_matrix, document_ids)
cosine_results_ranked = rank_results(cosine_results)

# Búsqueda con similitud Jaccard
jaccard_results = search_query_jaccard(preprocessed_query, documents, document_ids)
jaccard_results_ranked = rank_results(jaccard_results, method="jaccard")

In [33]:
# Mostrar resultados
print("Resultados con Cosine Similarity:", cosine_results_ranked, "\n")
print("Resultados con Jaccard Similarity:", jaccard_results_ranked)

Resultados con Cosine Similarity: [('10505', 0.34892367794324897), ('20005', 0.3299156650054218), ('5258', 0.308162194328649), ('17568', 0.2954358975186243), ('10506', 0.29295814947970683), ('9450', 0.29264423630859265), ('1', 0.2859599728083888), ('15095', 0.28157317034293644), ('9953', 0.280059229341509), ('10760', 0.27270456669976495)] 

Resultados con Jaccard Similarity: [('10471', 0.1), ('10491', 0.1), ('21061', 0.1), ('6068', 0.09090909090909091), ('3190', 0.08333333333333333), ('8326', 0.07142857142857142), ('17733', 0.07142857142857142), ('18221', 0.07142857142857142), ('19358', 0.07142857142857142), ('6873', 0.045454545454545456)]


## 2.6. Evaluación del Sistema
### Objetivo: Medir la efectividad del sistema.
- Tareas:
    - Definir un conjunto de métricas de evaluación (precisión, recall, F1-score).
    - Realizar pruebas utilizando el conjunto de prueba del corpus.
    - Comparar el rendimiento de diferentes configuraciones del sistema.
    - Documentar los resultados y análisis.

In [34]:
# Vectorización TF-IDF
def vectorizar_tfidf(documentos):
    """Vectorizar los datos usando TF-IDF."""
    corpus = documentos["Contenido_Preprocesado"].tolist()  # Accede a la columna directamente
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
    return tfidf_vectorizer, tfidf_matrix

# Vectorización Bag of Words
def vectorizar_bow(documentos):
    """Vectorizar los datos usando Bag of Words."""
    corpus = documentos["Contenido_Preprocesado"].tolist()
    bow_vectorizer = CountVectorizer()
    bow_matrix = bow_vectorizer.fit_transform(corpus)
    return bow_vectorizer, bow_matrix

# Función para convertir texto en un vector promedio usando Word2Vec
def get_average_word2vec_vector(text, model):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Obtener documentos relevantes desde el índice invertido
def obtener_documentos_relevantes(query_terms, indice_invertido):
    relevant_docs = set()
    for term in query_terms:
        term_docs = indice_invertido[indice_invertido["Término"] == term]["Documentos"]
        if term_docs.empty:
            print(f"Término no encontrado en el índice invertido: {term}")  # Depuración
        else:
            for doc_list in term_docs:
                try:
                    doc_ids = list(map(int, doc_list.split(',')))
                    relevant_docs.update(doc_ids)
                except ValueError:
                    continue
    return relevant_docs


In [35]:
# Función de preprocesamiento sin stemming
def preprocess_text_without_stemming(content):
    """Realiza limpieza, tokenización, eliminación de stopwords y normalización."""
    # Limpieza del texto
    cleaned_text = clean_text(content)
    # Tokenización
    tokens = word_tokenize(cleaned_text)
    # Eliminación de stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Reconstrucción del texto preprocesado
    preprocessed_text = " ".join(tokens)
    return preprocessed_text

In [36]:
# Cargar datos preprocesados
documentos = pd.read_excel("../data/reuters/reuters_data_preprocessed.xlsx")
indice_invertido = pd.read_excel("../data/reuters/inverted_index.xlsx")

In [37]:
# Vectorización
tfidf_vectorizer, tfidf_matrix = vectorizar_tfidf(documentos)
bow_vectorizer, bow_matrix = vectorizar_bow(documentos)

# Tokenización y Word2Vec
corpus = documentos["Contenido_Preprocesado"].tolist()
tokenized_corpus = [doc.split() for doc in corpus]
word2vec_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1)
corpus_vectors = [get_average_word2vec_vector(doc, word2vec_model) for doc in corpus]


In [38]:
# Guardar TF-IDF Matrix en CSV
def save_tfidf_matrix_csv(tfidf_matrix, tfidf_vectorizer, filename="tfidf_matrix.csv"):
    """Guarda la matriz TF-IDF en un archivo CSV."""
    tfidf_df = pd.DataFrame(
        tfidf_matrix.toarray(), 
        columns=tfidf_vectorizer.get_feature_names_out()
    )
    tfidf_df.to_csv(filename, index=False)
    print(f"Matriz TF-IDF guardada en: {filename}")

# Guardar Bag of Words Matrix en CSV
def save_bow_matrix_csv(bow_matrix, bow_vectorizer, filename="bow_matrix.csv"):
    """Guarda la matriz BoW en un archivo CSV."""
    bow_df = pd.DataFrame(
        bow_matrix.toarray(), 
        columns=bow_vectorizer.get_feature_names_out()
    )
    bow_df.to_csv(filename, index=False)
    print(f"Matriz Bag of Words guardada en: {filename}")

# Guardar Word2Vec Corpus Vectors en CSV
def save_corpus_vectors_csv(corpus_vectors, filename="corpus_vectors.csv"):
    """Guarda los vectores del corpus generados con Word2Vec en un archivo CSV."""
    corpus_df = pd.DataFrame(corpus_vectors)
    corpus_df.to_csv(filename, index=False)
    print(f"Vectores del corpus (Word2Vec) guardados en: {filename}")

In [39]:
# Llamar a las funciones para guardar
save_tfidf_matrix_csv(tfidf_matrix, tfidf_vectorizer, "../data/reuters/tfidf_matrix.csv")
save_bow_matrix_csv(bow_matrix, bow_vectorizer, "../data/reuters/bow_matrix.csv")
save_corpus_vectors_csv(corpus_vectors, "../data/reuters/corpus_vectors.csv")

Matriz TF-IDF guardada en: ../data/reuters/tfidf_matrix.csv
Matriz Bag of Words guardada en: ../data/reuters/bow_matrix.csv
Vectores del corpus (Word2Vec) guardados en: ../data/reuters/corpus_vectors.csv


In [40]:
# Simulación de consulta
query = "trade cocoa"
method = "bow"

In [41]:
preprocessed_query = preprocess_text_without_stemming(query)

In [42]:
# Vectorización y cálculo de similitud
if method == 'tfidf':
    query_vector = tfidf_vectorizer.transform([preprocessed_query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
elif method == 'bow':
    query_vector = bow_vectorizer.transform([preprocessed_query])
    similarities = cosine_similarity(query_vector, bow_matrix).flatten()
elif method == 'word2vec':
    query_vector = get_average_word2vec_vector(preprocessed_query, word2vec_model)
    similarities = cosine_similarity([query_vector], corpus_vectors).flatten()

In [43]:
# Seleccionar documentos recuperados (similitud >= 0.001)
retrieved_indices = [idx for idx, sim in enumerate(similarities) if sim >= 0.000001]
retrieved_docs = {int(documentos.iloc[idx]['Nombre']) for idx in retrieved_indices}  # Ajustar al formato correcto

query_terms = preprocessed_query.split()  # Términos preprocesados
true_positive_docs = obtener_documentos_relevantes(query_terms, indice_invertido)

# Logs para depuración
print("Query Terms:", query_terms)
print("Documentos verdaderos (True Positives):", true_positive_docs)
print("Documentos recuperados:", retrieved_docs)

Query Terms: ['trade', 'cocoa']
Documentos verdaderos (True Positives): {1, 8193, 8198, 8200, 8206, 8212, 45, 47, 8240, 16433, 16434, 8244, 16442, 59, 8252, 8273, 100, 8299, 121, 8314, 16505, 8326, 16521, 8342, 8344, 8349, 180, 16565, 16564, 8374, 8396, 209, 220, 8416, 237, 242, 16627, 16630, 247, 8440, 16636, 8453, 271, 16656, 275, 297, 314, 8506, 320, 323, 338, 16723, 342, 354, 356, 16745, 8554, 16752, 8563, 16759, 16762, 16763, 16766, 16774, 16775, 16777, 8592, 16784, 8595, 16787, 16788, 16790, 8599, 8596, 8597, 8602, 16794, 8606, 8608, 8615, 425, 8618, 8621, 8635, 8637, 8644, 16852, 16853, 8662, 8663, 8664, 16856, 8667, 8668, 475, 8671, 8676, 16871, 8692, 501, 8699, 8702, 8713, 522, 8714, 8725, 16926, 16929, 16932, 8746, 8747, 8748, 8761, 8802, 17036, 8849, 17042, 8850, 17044, 8853, 8861, 704, 706, 714, 718, 8933, 742, 8943, 8944, 8951, 768, 8961, 8978, 8982, 791, 798, 8998, 9007, 9015, 829, 9022, 9025, 833, 843, 849, 855, 856, 9055, 9060, 872, 875, 880, 9076, 890, 894, 9088, 897, 

In [44]:
# Seleccionar documentos recuperados (similitud > 0)
retrieved_indices = [idx for idx, sim in enumerate(similarities) if sim > 0]
retrieved_docs = set(idx for idx in retrieved_indices)

# Obtener documentos relevantes del índice invertido

true_positive_docs = obtener_documentos_relevantes(query_terms, indice_invertido)

# Calcular métricas
if retrieved_docs and true_positive_docs:
    true_positives = len(retrieved_docs & true_positive_docs)
    false_positives = len(retrieved_docs - true_positive_docs)
    false_negatives = len(true_positive_docs - retrieved_docs)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
else:
    precision = recall = f1_score = 0
    
# Logs de métricas
print("True Positives encontrados:", len(true_positive_docs))
print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

True Positives encontrados: 1536
Precisión: 0.0788
Recall: 0.0788
F1-Score: 0.0788
